Author: Taylor Larkin \
Version Date: 2024-01-11

# Time Series Hierarchical Reconciliation

This AI Accelerator demonstrates how to reconcile (e.g., post-processing to sum appropriately) independent time series forecasts with a hierarchical structure. Reconciling, also known as making ["coherent" forecasts](https://otexts.com/fpp3/hierarchical.html), is often a requirement when submitting hierarchical forecasts to stakeholders. This notebook leverages the increasingly popular [HierarchicalForecast](https://nixtlaverse.nixtla.io/hierarchicalforecast/index.html) python library to do the reconciliation on forecasts generated from DataRobot time series deployments. The steps demonstrated are as follows:

0. Installing `hierarchicalforecast`
1. Importing libraries
2. Loading the example dataset
3. Preparing training data for each hierarchy
4. Building models for each level
5. Deploying models for each level
6. Making forecasts
7. Preparing the forecasts
8. Reconcile forecasts
9. Comparing forecasts
10. Conclusion

Note that steps 2-6 steps are purely for providing example time series deployments and forecasts from those deployments (in case you don't have any). If you already have a set of forecasts you want to reconcile, feel free to skip to step 7.

### 0. Install `hierarchicalforecast`

As a preliminary, you will install (if not already done so) the aforementioned package. See the [package documentation](https://nixtlaverse.nixtla.io/hierarchicalforecast/index.html#pypi) for more details. Once this is installed, restart the kernel so that this library can be available.

In [ ]:
# !pip install hierarchicalforecast==0.4.1

### 1. Importing libraries

[Read](https://docs.datarobot.com/en/docs/api/api-quickstart/index.html) about different options for connecting to DataRobot from the client. Load the remaining libraries below in the usual way.

In [57]:
# Establish connection
import datarobot as dr

print(f"DataRobot version: {dr.__version__}")
dr.Client()

DataRobot version: 3.2.0


In [60]:
from io import StringIO

from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, MinTrace, TopDown
from hierarchicalforecast.utils import aggregate
import numpy as np
import pandas as pd

### 2. Loading a dataset

The example dataset you'll use will come from the [Google Cloud Platforms COVID-19 Open Data repository](https://github.com/GoogleCloudPlatform/covid-19-open-data/blob/main/docs/table-hospitalizations.md). This particular dataset has various COVID-19 hospitalizations statistics over time for various geographic regions, including state-level data for the United States. To make the problem more interesting, you'll add two more granularities (region and division) using the file found [here](https://github.com/cphalpert/census-regions/blob/master/us%20census%20bureau%20regions%20and%20divisions.csv). The end result will be a dataset containing the count of COVID-19 cases hospitalized after a positive test at the state-level (which will subsequently be aggregated to the higher-levels).

In [61]:
# Load data
# https://github.com/GoogleCloudPlatform/covid-19-open-data/blob/main/docs/table-hospitalizations.md
df = pd.read_csv(
    "https://storage.googleapis.com/covid19-open-data/v3/hospitalizations.csv",
    parse_dates=["date"],
    usecols=[
        "date",
        "location_key",
        "current_hospitalized_patients",
    ],
)
df

,date,location_key,current_hospitalized_patients
0,0022-01-10,AR,NaN
1,0022-01-20,AR,NaN
2,0202-03-30,AR,NaN
3,0221-07-06,AR,NaN
4,1202-01-07,AR,NaN
...,...,...,...
1768480,2022-09-11,US_WY,13.0
1768481,2022-09-12,US_WY,16.0
1768482,2022-09-13,US_WY,16.0
1768483,2022-09-14,US_WY,22.0


In [62]:
# Subset to just US state-level data as our lowest granularity
df = df.loc[
    (df["location_key"].str.startswith("US_")) & (df["location_key"].str.len() == 5)
].reset_index(drop=True)
df

,date,location_key,current_hospitalized_patients
0,2020-03-06,US_AK,NaN
1,2020-03-07,US_AK,NaN
2,2020-03-08,US_AK,NaN
3,2020-03-09,US_AK,NaN
4,2020-03-10,US_AK,NaN
...,...,...,...
50694,2022-09-11,US_WY,13.0
50695,2022-09-12,US_WY,16.0
50696,2022-09-13,US_WY,16.0
50697,2022-09-14,US_WY,22.0


In [63]:
# Load regions data
df_regions = pd.read_csv(
    "https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv"
)

# Add and order by hierarchy
df_regions["Country"] = "US"
df_regions = df_regions[["Country", "Region", "Division", "State", "State Code"]]

# Add regions data
state_split = df["location_key"].str.split("_", expand=True)
df["State Code"] = state_split[1]
df = df_regions.merge(df, how="left", on="State Code")

# Drop redundant columns
df = df.drop(["State Code", "location_key"], axis=1)

# Drop missing target values
df = df.loc[~df["current_hospitalized_patients"].isnull()].reset_index(drop=True)

# Convert date to a datetime
df["date"] = pd.to_datetime(df["date"])

# Check
df

,Country,Region,Division,State,date,current_hospitalized_patients
0,US,West,Pacific,Alaska,2020-04-18,39.0
1,US,West,Pacific,Alaska,2020-04-19,37.0
2,US,West,Pacific,Alaska,2020-04-20,46.0
3,US,West,Pacific,Alaska,2020-04-21,42.0
4,US,West,Pacific,Alaska,2020-04-22,39.0
...,...,...,...,...,...,...
45130,US,West,Mountain,Wyoming,2022-09-11,13.0
45131,US,West,Mountain,Wyoming,2022-09-12,16.0
45132,US,West,Mountain,Wyoming,2022-09-13,16.0
45133,US,West,Mountain,Wyoming,2022-09-14,22.0


### 3. Preparing training data for each hierarchy

The dataset above will serve as your lowest granularity (or lowest-level hierarchy). From here, you'll create additional training datasets at the division-level, region-level, and country-level. This means that you'll be building four DataRobot projects. To help keep things organized, you'll create a nested dictionary that houses information around each of these hierarchies needed for training dataset preparation and modeling.

In [65]:
# Organize information in a dictionary
hierarchy_info = {
    "Country-level": {
        "multiseries_id_column": None,
        "grouping_columns": ["Country", "date"],
    },
    "Region-level": {
        "multiseries_id_column": ["Region"],
        "grouping_columns": ["Country", "Region", "date"],
    },
    "Division-level": {
        "multiseries_id_column": ["Division"],
        "grouping_columns": ["Country", "Region", "Division", "date"],
    },
    "State-level": {
        "multiseries_id_column": ["State"],
        "grouping_columns": ["Country", "Region", "Division", "State", "date"],
    },
}

In [66]:
# Check info
hierarchy_info

{'Country-level': {'multiseries_id_column': None,
  'grouping_columns': ['Country', 'date']},
 'Region-level': {'multiseries_id_column': ['Region'],
  'grouping_columns': ['Country', 'Region', 'date']},
 'Division-level': {'multiseries_id_column': ['Division'],
  'grouping_columns': ['Country', 'Region', 'Division', 'date']},
 'State-level': {'multiseries_id_column': ['State'],
  'grouping_columns': ['Country', 'Region', 'Division', 'State', 'date']}}

### 4. Building models for each level

In this section, you'll build models for each hierarchy forecasting the next 1-7 days out. Note that you'll also remove the last week's worth of data, which will be used as a blind holdout later in the notebook to compare different reconciliation methods. The benefit of using DataRobot for this step is the platform will try a plethora of approaches, both classical and more modern-day machine-learning based ones to ensure you have the best model for the job.

In [67]:
# Split data into training and testing
# Using last week as a blind holdout
df_train = df.loc[df["date"] <= (df["date"].max() - pd.Timedelta("7d"))]
df_train

,Country,Region,Division,State,date,current_hospitalized_patients
0,US,West,Pacific,Alaska,2020-04-18,39.0
1,US,West,Pacific,Alaska,2020-04-19,37.0
2,US,West,Pacific,Alaska,2020-04-20,46.0
3,US,West,Pacific,Alaska,2020-04-21,42.0
4,US,West,Pacific,Alaska,2020-04-22,39.0
...,...,...,...,...,...,...
45123,US,West,Mountain,Wyoming,2022-09-04,18.0
45124,US,West,Mountain,Wyoming,2022-09-05,18.0
45125,US,West,Mountain,Wyoming,2022-09-06,16.0
45126,US,West,Mountain,Wyoming,2022-09-07,13.0


In [69]:
# Build projects for each
for key in hierarchy_info.keys():
    # Create training data (summing to the hierarchical-level)
    df_train_hier = (
        df_train.groupby(hierarchy_info[key]["grouping_columns"])
        .sum(numeric_only=True)
        .reset_index()
    )

    # Creating project
    print(f"Creating project and running models for {key} forecasting...\n")
    project = dr.Project.create(
        sourcedata=df_train_hier,
        project_name=f"{key} forecasting model",
    )

    # Create the feature list
    fl = project.create_featurelist(
        name="Predictors",
        features=["date", "current_hospitalized_patients"],
    )

    # Run models
    project = project.analyze_and_model(
        target="current_hospitalized_patients",
        worker_count=-1,
        partitioning_method=dr.DatetimePartitioningSpecification(
            use_time_series=True,
            datetime_partition_column="date",
            multiseries_id_columns=hierarchy_info[key]["multiseries_id_column"],
            forecast_window_start=1,
            forecast_window_end=7,
        ),
        featurelist_id=fl.id,
    )

    # Save project
    hierarchy_info[key]["project"] = project

# Wait for them to finish
for key in hierarchy_info.keys():
    # Wait for them to complete
    hierarchy_info[key]["project"].wait_for_autopilot(check_interval=60)

print("\nModel building finished!\n")

Creating project and running models for Country-level forecasting...

Creating project and running models for Region-level forecasting...

Creating project and running models for Division-level forecasting...

Creating project and running models for State-level forecasting...

In progress: 0, queued: 0 (waited: 0s)
In progress: 0, queued: 0 (waited: 0s)
In progress: 1, queued: 0 (waited: 0s)
In progress: 1, queued: 0 (waited: 1s)
In progress: 1, queued: 0 (waited: 1s)
In progress: 1, queued: 0 (waited: 2s)
In progress: 1, queued: 0 (waited: 4s)
In progress: 1, queued: 0 (waited: 6s)
In progress: 0, queued: 0 (waited: 9s)
In progress: 0, queued: 0 (waited: 16s)
In progress: 0, queued: 0 (waited: 29s)
In progress: 0, queued: 0 (waited: 55s)
In progress: 0, queued: 0 (waited: 107s)
In progress: 4, queued: 0 (waited: 0s)
In progress: 4, queued: 0 (waited: 1s)
In progress: 4, queued: 0 (waited: 1s)
In progress: 4, queued: 0 (waited: 2s)
In progress: 2, queued: 0 (waited: 3s)
In progress: 2,

In [119]:
# Inspect best model for each hierarchy
# Note that the same model doesn't dominate all hierarchies (aka no free lunch)
for key in hierarchy_info.keys():
    # Get recommended model
    model = dr.ModelRecommendation.get(project_id=hierarchy_info[key]["project"].id).get_model()

    # Save model
    hierarchy_info[key]["model"] = model

    print(f"Best model for {key}: {model.model_type}\n")

Best model for Country-level: Temporal Hierarchical Model with Elastic Net and XGBoost

Best model for Region-level: eXtreme Gradient Boosting on ElasticNet Predictions (learning rate =0.3)

Best model for Division-level: Light Gradient Boosting on ElasticNet Predictions (learning rate =0.3) 

Best model for State-level: Light Gradient Boosting on ElasticNet Predictions (learning rate =0.3) 



### 5. Deploying models for each level

Here, you'll deploy the recommended model for each project, saving the deployment in the nested dictionary of hierarchical information.

In [71]:
# List prediction server to use
prediction_server = dr.PredictionServer.list()[0]

# Cycle through and deploy models
for key in hierarchy_info.keys():
    # Deploy
    print(f"Deploying {key} model...\n")
    deployment = dr.Deployment.create_from_learning_model(
        model_id=hierarchy_info[key]["model"].id,
        label=f"{key} TS Model",
        description=f"{key} forecasting model for Time Series Hierarchical Reconciliation AI Accelerator",
        default_prediction_server_id=prediction_server.id,
        max_wait=60 * 60 * 24,
    )

    # Save deployment
    hierarchy_info[key]["deployment"] = deployment

In [72]:
# Check info
hierarchy_info

{'Country-level': {'multiseries_id_column': None,
  'grouping_columns': ['Country', 'date'],
  'project': Project(Country-level forecasting model),
  'model': DatetimeModel('Temporal Hierarchical Model with Elastic Net and XGBoost'),
  'deployment': Deployment(Country-level TS Model)},
 'Region-level': {'multiseries_id_column': ['Region'],
  'grouping_columns': ['Country', 'Region', 'date'],
  'project': Project(Region-level forecasting model),
  'model': DatetimeModel('eXtreme Gradient Boosting on ElasticNet Predictions (learning rate =0.3)'),
  'deployment': Deployment(Region-level TS Model)},
 'Division-level': {'multiseries_id_column': ['Division'],
  'grouping_columns': ['Country', 'Region', 'Division', 'date'],
  'project': Project(Division-level forecasting model),
  'model': DatetimeModel('Light Gradient Boosting on ElasticNet Predictions (learning rate =0.3) '),
  'deployment': Deployment(Division-level TS Model)},
 'State-level': {'multiseries_id_column': ['State'],
  'groupi

### 6. Make forecasts

Next, you'll leverage DataRobot batch prediction API to generate forecasts and load them into memory. Note you'll also pass the grouping columns via the `passthrough_columns` argument. This is important, as it will allow us to more easily augment the dataframe of forecasts into a format ready for `hierarchicalforecast`.

In [73]:
# Define the forecast point
forecast_point = df_train["date"].max()
forecast_point

Timestamp('2022-09-08 00:00:00')

In [94]:
# Collection predictions for each hierarchy
df_forecasts = pd.DataFrame()
for key in hierarchy_info.keys():
    print(f"Starting batch prediction job for {key} forecasting model...")

    ### Prep prediction data ###
    df_test = (
        df.groupby(hierarchy_info[key]["grouping_columns"]).sum(numeric_only=True).reset_index()
    ).sort_values(hierarchy_info[key]["grouping_columns"])

    # Score prediction data
    job = dr.BatchPredictionJob.score(
        deployment=hierarchy_info[key]["deployment"],
        intake_settings={
            "type": "localFile",
            "file": df_test,
        },
        timeseries_settings={
            "type": "forecast",
            "forecast_point": forecast_point,
        },
        output_settings={
            "type": "localFile",
        },
        passthrough_columns=hierarchy_info[key]["grouping_columns"],
    )

    # Returning response as a dataframe and appending
    df_forecasts = pd.concat(
        [
            df_forecasts,
            pd.read_csv(StringIO(job.get_result_when_complete(max_wait=24 * 24 * 60).decode())),
        ],
        ignore_index=True,
    )


# Renaming for simplicity
df_forecasts = df_forecasts.rename(
    columns={"current_hospitalized_patients (actual)_PREDICTION": "PREDICTION"}
)

# Check
df_forecasts

Starting batch prediction job for Country-level forecasting model...
Starting batch prediction job for Region-level forecasting model...
Starting batch prediction job for Division-level forecasting model...
Starting batch prediction job for State-level forecasting model...


,FORECAST_POINT,date,FORECAST_DISTANCE,PREDICTION,DEPLOYMENT_APPROVAL_STATUS,Country,Region,Division,State
0,2022-09-08T00:00:00Z,2022-09-09T00:00:00.000000Z,1,30167.827454,APPROVED,US,NaN,NaN,NaN
1,2022-09-08T00:00:00Z,2022-09-10T00:00:00.000000Z,2,30168.619727,APPROVED,US,NaN,NaN,NaN
2,2022-09-08T00:00:00Z,2022-09-11T00:00:00.000000Z,3,30153.126692,APPROVED,US,NaN,NaN,NaN
3,2022-09-08T00:00:00Z,2022-09-12T00:00:00.000000Z,4,30153.918537,APPROVED,US,NaN,NaN,NaN
4,2022-09-08T00:00:00Z,2022-09-13T00:00:00.000000Z,5,30154.710383,APPROVED,US,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
450,2022-09-08T00:00:00Z,2022-09-11T00:00:00.000000Z,3,430.331571,APPROVED,US,West,Pacific,Washington
451,2022-09-08T00:00:00Z,2022-09-12T00:00:00.000000Z,4,428.037839,APPROVED,US,West,Pacific,Washington
452,2022-09-08T00:00:00Z,2022-09-13T00:00:00.000000Z,5,426.734262,APPROVED,US,West,Pacific,Washington
453,2022-09-08T00:00:00Z,2022-09-14T00:00:00.000000Z,6,426.705117,APPROVED,US,West,Pacific,Washington


### 7. Preparing the forecasts

Perhaps the most important section of this accelerator is converting your forecasts into the [correct format](http://www.sktime.net/en/latest/examples/AA_datatypes_and_datasets.html?highlight=multi#Section-1.3:-Hierarchical-time-series---the-%22Hierarchical%22-scitypefrom) to be used in `hierarchicalforecast`. For hierarchical reconciliation, this requires 

- the forecasts for each level to be stacked on top of one another
- a date column called "ds", 
- an index that labels the hierarchy
- a target column called "y" (if providing in-sample forecasts)

To make this easier, a function is provided below to help you get the data into the right format. Note that it provides a tuple of outputs, which include

1. The correctly formatted dataframe of forecasts
2. A dataframe representing the [aggregation constrains matrix](https://nixtlaverse.nixtla.io/hierarchicalforecast/utils.html)
3. A dictionary that maps the hierarchical levels to one another

Each of these are necessary to pass to the associated reconciliation function. Additionally, for some reconciliation methods, they require providing in-sample forecasts and target values. To this end, you will use DataRobot's [historical forecasts functionality](https://docs.datarobot.com/en/docs/api/reference/batch-prediction-api/batch-pred-ts.html), which provides multiple-step ahead forecasts for a range of dates.

In [95]:
def prep_data_for_reconciliation(
    df: pd.DataFrame,
    spec: [[str]],
    date_column: str,
    prediction_column: str = None,
    target_column: str = None,
) -> (pd.DataFrame, pd.DataFrame, dict):
    """
    Function to prepare forecasting data for reconciliation using hierarchicalforecast package
    (inspired by the package's "aggregate" function)

    Parameters
    ----------
    df: dataframe of forecasts with each hierarchy represented as columns (with missing values in hierachy columns)
    spec: list of lists specifying the hierarchy
    date_column: name of main datetime column
    prediction_column: name of column containing forecasted values
    target_column: name of target column -- if supplied, will be added to output as "y"

    Returns
    -------
    Objects necessary for passing into HierarchicalReconciliation().reconcile()

    """
    # Copy
    df = df.copy(deep=True)

    # Renaming to match hierarhicalforecast's expectation
    df = df.rename(
        columns={
            date_column: "ds",
            target_column: "y",
        }
    )

    # Gather columns
    col_names = df.columns.tolist()
    supplied_cols = [
        x
        for x in [
            "ds",
            "y",
            prediction_column,
        ]
        if x in col_names
    ]

    # Get lowest-level hierarchy
    max_len_idx = np.argmax([len(hier) for hier in spec])
    lowest_hier = spec[max_len_idx]

    # Combine them into one column
    df["unique_id"] = df[lowest_hier].apply(lambda x: "/".join(x.dropna()), axis=1)

    # Sort them from highest to lowest hierachy
    df["hierarchy_missing_count"] = df[lowest_hier].isnull().sum(axis=1)
    df = df.sort_values(
        ["hierarchy_missing_count", "unique_id", "ds"], ascending=[False, True, True]
    )

    # Get lowest-level forecasts
    Y_lowest = df.loc[df["hierarchy_missing_count"] == df["hierarchy_missing_count"].min()]

    # Will reuse the "aggregate" function to shortcut our way to getting the summation matrix and tags
    _, S, tags = aggregate(
        df=Y_lowest.rename(columns={prediction_column: "y"}), spec=spec, sparse_s=False
    )

    # Subset
    Y_df = df[["unique_id"] + supplied_cols].set_index("unique_id")

    return Y_df, S, tags

In [96]:
# Define hierarchy levels
hierarchy_levels = [
    ["Country"],
    ["Country", "Region"],
    ["Country", "Region", "Division"],
    ["Country", "Region", "Division", "State"],
]

In [97]:
# Apply prep function
df_forecasts_prepped, summation_matrix, hierarchy_tags = prep_data_for_reconciliation(
    df=df_forecasts,
    spec=hierarchy_levels,
    date_column="date",
    prediction_column="PREDICTION",
    target_column=None,
)

# Check prepped forecasts
df_forecasts_prepped

,ds,PREDICTION
unique_id,,
US,2022-09-09T00:00:00.000000Z,30167.827454
US,2022-09-10T00:00:00.000000Z,30168.619727
US,2022-09-11T00:00:00.000000Z,30153.126692
US,2022-09-12T00:00:00.000000Z,30153.918537
US,2022-09-13T00:00:00.000000Z,30154.710383
...,...,...
US/West/Pacific/Washington,2022-09-11T00:00:00.000000Z,430.331571
US/West/Pacific/Washington,2022-09-12T00:00:00.000000Z,428.037839
US/West/Pacific/Washington,2022-09-13T00:00:00.000000Z,426.734262


In [98]:
# Summing matrix
summation_matrix

,US/Midwest/East North Central/Illinois,US/Midwest/East North Central/Indiana,US/Midwest/East North Central/Michigan,US/Midwest/East North Central/Ohio,US/Midwest/East North Central/Wisconsin,US/Midwest/West North Central/Iowa,US/Midwest/West North Central/Kansas,US/Midwest/West North Central/Minnesota,US/Midwest/West North Central/Missouri,US/Midwest/West North Central/Nebraska,...,US/West/Mountain/Montana,US/West/Mountain/Nevada,US/West/Mountain/New Mexico,US/West/Mountain/Utah,US/West/Mountain/Wyoming,US/West/Pacific/Alaska,US/West/Pacific/California,US/West/Pacific/Hawaii,US/West/Pacific/Oregon,US/West/Pacific/Washington
US,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
US/Midwest,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
US/Northeast,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
US/South,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
US/West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
US/West/Pacific/Alaska,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
US/West/Pacific/California,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
US/West/Pacific/Hawaii,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
US/West/Pacific/Oregon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [99]:
# Tags
hierarchy_tags

{'Country': array(['US'], dtype=object),
 'Country/Region': array(['US/Midwest', 'US/Northeast', 'US/South', 'US/West'], dtype=object),
 'Country/Region/Division': array(['US/Midwest/East North Central', 'US/Midwest/West North Central',
        'US/Northeast/Middle Atlantic', 'US/Northeast/New England',
        'US/South/East South Central', 'US/South/South Atlantic',
        'US/South/West South Central', 'US/West/Mountain',
        'US/West/Pacific'], dtype=object),
 'Country/Region/Division/State': array(['US/Midwest/East North Central/Illinois',
        'US/Midwest/East North Central/Indiana',
        'US/Midwest/East North Central/Michigan',
        'US/Midwest/East North Central/Ohio',
        'US/Midwest/East North Central/Wisconsin',
        'US/Midwest/West North Central/Iowa',
        'US/Midwest/West North Central/Kansas',
        'US/Midwest/West North Central/Minnesota',
        'US/Midwest/West North Central/Missouri',
        'US/Midwest/West North Central/Nebraska',
   

In [100]:
# Obtaining in-sample forecasts (some methods below require them)
df_insample_forecasts = pd.DataFrame()
for key in hierarchy_info.keys():
    print(f"Starting batch prediction job for {key} forecasting model...")

    ### Prep prediction data ###
    df_test = (
        df.groupby(hierarchy_info[key]["grouping_columns"]).sum(numeric_only=True).reset_index()
    ).sort_values(hierarchy_info[key]["grouping_columns"])

    ### Score prediction data ###
    job = dr.BatchPredictionJob.score(
        deployment=hierarchy_info[key]["deployment"],
        intake_settings={
            "type": "localFile",
            "file": df_test,
        },
        # Using last year's worth of data
        timeseries_settings={
            "type": "historical",
            "predictions_start_date": pd.to_datetime(forecast_point) - pd.Timedelta("365D"),
            "predictions_end_date": forecast_point,
        },
        output_settings={
            "type": "localFile",
        },
        # Ensuring target is in there
        passthrough_columns=hierarchy_info[key]["grouping_columns"]
        + ["current_hospitalized_patients"],
    )

    # Returning response as a dataframe and appending
    df_insample_forecasts = pd.concat(
        [
            df_insample_forecasts,
            pd.read_csv(StringIO(job.get_result_when_complete(max_wait=24 * 24 * 60).decode())),
        ],
        ignore_index=True,
    )


# Renaming for simplicity
df_insample_forecasts = df_insample_forecasts.rename(
    columns={"current_hospitalized_patients (actual)_PREDICTION": "PREDICTION"}
)

# Since DataRobot generating predictions for each forecast distance, we'll just use one-step ahead forecasts
df_insample_forecasts = df_insample_forecasts.loc[
    df_insample_forecasts["FORECAST_DISTANCE"] == df_insample_forecasts["FORECAST_DISTANCE"].min()
].reset_index(drop=True)

# Check
df_insample_forecasts

Starting batch prediction job for Country-level forecasting model...
Starting batch prediction job for Region-level forecasting model...
Starting batch prediction job for Division-level forecasting model...
Starting batch prediction job for State-level forecasting model...


,FORECAST_POINT,date,FORECAST_DISTANCE,PREDICTION,DEPLOYMENT_APPROVAL_STATUS,Country,current_hospitalized_patients,Region,Division,State
0,2021-09-07T00:00:00Z,2021-09-08T00:00:00.000000Z,1,93372.299143,APPROVED,US,96102.0,NaN,NaN,NaN
1,2021-09-08T00:00:00Z,2021-09-09T00:00:00.000000Z,1,94993.032308,APPROVED,US,95335.0,NaN,NaN,NaN
2,2021-09-09T00:00:00Z,2021-09-10T00:00:00.000000Z,1,95441.634450,APPROVED,US,94216.0,NaN,NaN,NaN
3,2021-09-10T00:00:00Z,2021-09-11T00:00:00.000000Z,1,93125.676389,APPROVED,US,91645.0,NaN,NaN,NaN
4,2021-09-11T00:00:00Z,2021-09-12T00:00:00.000000Z,1,89296.079668,APPROVED,US,91205.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
23720,2022-09-02T00:00:00Z,2022-09-03T00:00:00.000000Z,1,485.708375,APPROVED,US,476.0,West,Pacific,Washington
23721,2022-09-03T00:00:00Z,2022-09-04T00:00:00.000000Z,1,481.060394,APPROVED,US,458.0,West,Pacific,Washington
23722,2022-09-04T00:00:00Z,2022-09-05T00:00:00.000000Z,1,469.677556,APPROVED,US,450.0,West,Pacific,Washington
23723,2022-09-05T00:00:00Z,2022-09-06T00:00:00.000000Z,1,450.141100,APPROVED,US,452.0,West,Pacific,Washington


In [101]:
# Now prep the insample predictions in the same way
df_insample_forecasts_prepped, _, _ = prep_data_for_reconciliation(
    df=df_insample_forecasts,
    spec=hierarchy_levels,
    date_column="date",
    prediction_column="PREDICTION",
    target_column="current_hospitalized_patients",
)

# Check prepped insample forecasts
df_insample_forecasts_prepped

,ds,y,PREDICTION
unique_id,,,
US,2021-09-08T00:00:00.000000Z,96102.0,93372.299143
US,2021-09-09T00:00:00.000000Z,95335.0,94993.032308
US,2021-09-10T00:00:00.000000Z,94216.0,95441.634450
US,2021-09-11T00:00:00.000000Z,91645.0,93125.676389
US,2021-09-12T00:00:00.000000Z,91205.0,89296.079668
...,...,...,...
US/West/Pacific/Washington,2022-09-03T00:00:00.000000Z,476.0,485.708375
US/West/Pacific/Washington,2022-09-04T00:00:00.000000Z,458.0,481.060394
US/West/Pacific/Washington,2022-09-05T00:00:00.000000Z,450.0,469.677556


### 8. Reconciling forecasts

Below demonstrates the use of the class [HierarchicalReconciliation()](https://nixtlaverse.nixtla.io/hierarchicalforecast/core.html) to do the actual reconciliation. In the initialization step, you will specify the [methods](https://nixtlaverse.nixtla.io/hierarchicalforecast/methods.html) to use. Each have different strategies for making the forecasts coherent, with advantages and disadvantages. Note that for the `MinTrace()` methods, you'll specify `nonnegataive=True`, which will ensure the reconciled forecasts are >= 0.

In [155]:
# Specify reconciliation methods
reconcilers = [
    BottomUp(),
    TopDown(method="average_proportions"),
    MinTrace(method="ols", nonnegative=True),
    MinTrace(method="wls_struct", nonnegative=True),
    MinTrace(method="wls_var", nonnegative=True),
    MinTrace(method="mint_shrink", nonnegative=True),
    MinTrace(method="mint_cov", nonnegative=True),
]

# Reconcile them
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
df_forecasts_reconciled = hrec.reconcile(
    Y_hat_df=df_forecasts_prepped,
    Y_df=df_insample_forecasts_prepped,
    S=summation_matrix,
    tags=hierarchy_tags,
)

# Inspect
df_forecasts_reconciled

,ds,PREDICTION,PREDICTION/BottomUp,PREDICTION/TopDown_method-average_proportions,PREDICTION/MinTrace_method-ols_nonnegative-True,PREDICTION/MinTrace_method-wls_struct_nonnegative-True,PREDICTION/MinTrace_method-wls_var_nonnegative-True,PREDICTION/MinTrace_method-mint_shrink_nonnegative-True,PREDICTION/MinTrace_method-mint_cov_nonnegative-True
unique_id,,,,,,,,,
US,2022-09-09T00:00:00.000000Z,30167.827454,29750.046875,30167.828178,29890.169922,29567.933594,29465.953125,29460.992188,29531.933594
US,2022-09-10T00:00:00.000000Z,30168.619727,29826.525391,30168.619193,29909.410156,29624.921875,29544.841797,29518.083984,29580.972656
US,2022-09-11T00:00:00.000000Z,30153.126692,29880.011719,30153.127006,29920.144531,29679.242188,29619.826172,29572.695312,29623.625000
US,2022-09-12T00:00:00.000000Z,30153.918537,29984.591797,30153.918021,29960.800781,29782.578125,29751.933594,29685.664062,29723.945312
US,2022-09-13T00:00:00.000000Z,30154.710383,30131.548828,30154.710990,29992.781250,29885.132812,29896.728516,29793.732422,29819.097656
...,...,...,...,...,...,...,...,...,...
US/West/Pacific/Washington,2022-09-11T00:00:00.000000Z,430.331571,430.331573,577.628847,433.896057,426.729645,427.290436,430.206573,430.142639
US/West/Pacific/Washington,2022-09-12T00:00:00.000000Z,428.037839,428.037842,577.644000,430.904144,424.762177,425.427063,429.358124,430.102844
US/West/Pacific/Washington,2022-09-13T00:00:00.000000Z,426.734262,426.734253,577.659190,427.792175,422.858795,424.125153,429.225433,430.431183


### 9. Comparing forecasts

When it comes to comparing the reconciled and original forecasts, you'll explore three areas:

1. Coherence (i.e., do the forecasts sum accordingly?)
2. Similarity (i.e., how similar are the reconciled forecasts to the original ones?)
3. Performance (i.e., which forecasts have the lowest error overall?)

While coherence is an objective characteristic, similarity is much more subjective (i.e., how similar does my reconciled forecast need to be to my original one?). It's a natural trade-off, as the original forecasts need to be augmented in order to make them sum accordingly, and each reconciliation method augments them differently. Hence, you'll likely want to try each method, comparing the reconciled forecasts to the original ones with empirical metrics and visualizations. Measuring performance can also be helpful on deciding which reconciliation method to go with, as sometimes they can be more optimal than the original ones for a given use case. For a full treatment and explanation of each reconciliation approach, including the pros and cons, we recommended checking out this [chapter](https://otexts.com/fpp3/hierarchical.html).

Below, you'll see that `PREDICTION` does not sum to the same value at each hierarchy while the reconciled ones do, while being very similar and even being more accurate in some cases.

### Check for coherence

In [156]:
# Checking the overall sum
# They may not sum exactly in the thousands place (see https://github.com/Nixtla/hierarchicalforecast/issues/159)
df_forecasts_reconciled.groupby(
    df_forecasts_reconciled.index.str.count("/").rename("hierarchy_level")
).sum(numeric_only=True)

,PREDICTION,PREDICTION/BottomUp,PREDICTION/TopDown_method-average_proportions,PREDICTION/MinTrace_method-ols_nonnegative-True,PREDICTION/MinTrace_method-wls_struct_nonnegative-True,PREDICTION/MinTrace_method-wls_var_nonnegative-True,PREDICTION/MinTrace_method-mint_shrink_nonnegative-True,PREDICTION/MinTrace_method-mint_cov_nonnegative-True
hierarchy_level,,,,,,,,
0,211090.651959,210392.296875,211090.652712,209724.75,208603.15625,208512.59375,207908.046875,208191.000000
1,205266.775391,210392.281250,211090.652712,209724.75,208603.15625,208512.59375,207908.046875,208190.984375
2,207662.940720,210392.281250,211090.652712,209724.75,208603.15625,208512.59375,207908.046875,208190.984375
3,210392.277795,210392.281250,211090.652712,209724.75,208603.15625,208512.59375,207908.046875,208190.984375


In [157]:
# Checking the sum in a single day
single_day = df_forecasts_reconciled.loc[
    df_forecasts_reconciled["ds"] == df_forecasts_reconciled["ds"].max()
]
single_day.groupby(single_day.index.str.count("/").rename("hierarchy_level")).sum(numeric_only=True)

,PREDICTION,PREDICTION/BottomUp,PREDICTION/TopDown_method-average_proportions,PREDICTION/MinTrace_method-ols_nonnegative-True,PREDICTION/MinTrace_method-wls_struct_nonnegative-True,PREDICTION/MinTrace_method-wls_var_nonnegative-True,PREDICTION/MinTrace_method-mint_shrink_nonnegative-True,PREDICTION/MinTrace_method-mint_cov_nonnegative-True
hierarchy_level,,,,,,,,
0,30136.946938,30506.808594,30136.947318,30031.460938,30079.835938,30192.056641,29984.152344,29996.650391
1,29568.269043,30506.804688,30136.947318,30031.455078,30079.833984,30192.054688,29984.152344,29996.650391
2,30107.317936,30506.804688,30136.947318,30031.455078,30079.833984,30192.056641,29984.150391,29996.652344
3,30506.803925,30506.804688,30136.947318,30031.455078,30079.833984,30192.056641,29984.150391,29996.650391


### Check for similarity

In [158]:
# Simply using correlation coefficient
df_forecasts_reconciled.corr(numeric_only=True).iloc[[0]].T

,PREDICTION
PREDICTION,1.000000
PREDICTION/BottomUp,0.999878
PREDICTION/TopDown_method-average_proportions,0.997545
PREDICTION/MinTrace_method-ols_nonnegative-True,0.999924
PREDICTION/MinTrace_method-wls_struct_nonnegative-True,0.999905
PREDICTION/MinTrace_method-wls_var_nonnegative-True,0.999885
PREDICTION/MinTrace_method-mint_shrink_nonnegative-True,0.999891
PREDICTION/MinTrace_method-mint_cov_nonnegative-True,0.999892


### Check performance

In [159]:
# Preparing the evaluation data in the same way
df_test_eval = pd.DataFrame()
for key in hierarchy_info.keys():
    ### Prep evaluation data ###
    df_evaluation = (
        df.loc[df["date"] > forecast_point]
        .groupby(hierarchy_info[key]["grouping_columns"])
        .sum(numeric_only=True)
        .reset_index()
    ).sort_values(hierarchy_info[key]["grouping_columns"])

    # Append
    df_test_eval = pd.concat([df_test_eval, df_evaluation])

# Check
df_test_eval_prepped, _, _ = prep_data_for_reconciliation(
    df=df_test_eval,
    spec=hierarchy_levels,
    date_column="date",
    prediction_column=None,
    target_column="current_hospitalized_patients",
)
df_test_eval_prepped

,ds,y
unique_id,,
US,2022-09-09,29568.0
US,2022-09-10,27702.0
US,2022-09-11,27784.0
US,2022-09-12,28105.0
US,2022-09-13,27539.0
...,...,...
US/West/Pacific/Washington,2022-09-11,404.0
US/West/Pacific/Washington,2022-09-12,413.0
US/West/Pacific/Washington,2022-09-13,407.0


In [160]:
# See https://nixtlaverse.nixtla.io/hierarchicalforecast/evaluation.html


def mae(y: np.array, y_hat: np.array) -> float:
    """
    Mean absolute error calculation

    Parameters
    ----------
    y: target values
    yhat: predicted values

    Returns
    -------
    Mean absolute error

    """
    return np.mean(np.abs(y - y_hat))


# Computing mean absolute scaled error with respect to the original forecast
evaluator = HierarchicalEvaluation(evaluators=[mae])
evaluation = evaluator.evaluate(
    Y_hat_df=df_forecasts_reconciled,
    Y_test_df=df_test_eval_prepped,
    tags=hierarchy_tags,
    benchmark="PREDICTION",
)

# Metrics per method and hierarahcy-level
evaluation.T

level,Overall,Country,Country/Region,Country/Region/Division,Country/Region/Division/State
metric,mae-scaled,mae-scaled,mae-scaled,mae-scaled,mae-scaled
PREDICTION,1.0,1.0,1.0,1.0,1.0
PREDICTION/BottomUp,1.099281,0.95669,1.366429,1.166559,1.0
PREDICTION/TopDown_method-average_proportions,1.441199,1.0,1.647119,1.656877,1.559244
PREDICTION/MinTrace_method-ols_nonnegative-True,1.048387,0.915291,1.3073,1.112822,0.947736
PREDICTION/MinTrace_method-wls_struct_nonnegative-True,0.980325,0.845742,1.211029,1.037408,0.906784
PREDICTION/MinTrace_method-wls_var_nonnegative-True,0.985161,0.852775,1.218008,1.040496,0.909384
PREDICTION/MinTrace_method-mint_shrink_nonnegative-True,0.943914,0.815897,1.165337,0.994637,0.875518
PREDICTION/MinTrace_method-mint_cov_nonnegative-True,0.956127,0.824646,1.182437,1.007098,0.887571


In [161]:
# Sort by overall error
evaluation_overall = evaluation.iloc[[0]].T.sort_values(("Overall", "mae-scaled"))
evaluation_overall

level,Overall
metric,mae-scaled
PREDICTION/MinTrace_method-mint_shrink_nonnegative-True,0.943914
PREDICTION/MinTrace_method-mint_cov_nonnegative-True,0.956127
PREDICTION/MinTrace_method-wls_struct_nonnegative-True,0.980325
PREDICTION/MinTrace_method-wls_var_nonnegative-True,0.985161
PREDICTION,1.0
PREDICTION/MinTrace_method-ols_nonnegative-True,1.048387
PREDICTION/BottomUp,1.099281
PREDICTION/TopDown_method-average_proportions,1.441199


### 10. Conclusion

This notebook demonstrates how one could build time series models specific to each hierarchy with the power of DataRobot as well as how to reconcile them in a coherent manner with the help of the `HierarchicalForecast` python package. Because there's no free lunch, you'll likely need to train several models and investigate different reconciliation approaches to find the best combination for your use case. Fortunately, because DataRobot automates the former and this package enables the latter, stakeholders can be rest assured that they're receiving high-quality, intuitive results. For more information about DataRobot's time series functionality, visit our [documentation](https://docs.datarobot.com/en/docs/modeling/time/index.html). For more information about the `HierarchicalForecast` package, see the following [working paper](https://arxiv.org/abs/2207.03517).